In [14]:
# Dependencies and Setup

import os
import json
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
# Data Source
# https://www.kaggle.com/nasa/meteorite-landings

In [3]:
# File to Load

meteorcsv_path = ("C:/Users/jariwalaj2/Desktop/Project-2-VisualizeMe/Data/meteorite-landings.csv")


# Read the meteorite data
csv_meteors = pd.read_csv(meteorcsv_path)
csv_meteors

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.216670, -113.000000)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.883330, -99.900000)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"
...,...,...,...,...,...,...,...,...,...,...
45711,Zillah 002,31356,Valid,Eucrite,172.0,Found,1990.0,29.03700,17.01850,"(29.037000, 17.018500)"
45712,Zinder,30409,Valid,"Pallasite, ungrouped",46.0,Found,1999.0,13.78333,8.96667,"(13.783330, 8.966670)"
45713,Zlin,30410,Valid,H4,3.3,Found,1939.0,49.25000,17.66667,"(49.250000, 17.666670)"
45714,Zubkovsky,31357,Valid,L6,2167.0,Found,2003.0,49.78917,41.50460,"(49.789170, 41.504600)"


In [4]:
# Drop unnecessary columns
del csv_meteors["nametype"]
csv_meteors.head()

,name,id,recclass,mass,fall,year,reclat,reclong,GeoLocation
0,Aachen,1,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
2,Abee,6,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.216670, -113.000000)"
3,Acapulco,10,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.883330, -99.900000)"
4,Achiras,370,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"


In [5]:
del csv_meteors["fall"]
csv_meteors.head()

,name,id,recclass,mass,year,reclat,reclong,GeoLocation
0,Aachen,1,L5,21.0,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,H6,720.0,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
2,Abee,6,EH4,107000.0,1952.0,54.21667,-113.00000,"(54.216670, -113.000000)"
3,Acapulco,10,Acapulcoite,1914.0,1976.0,16.88333,-99.90000,"(16.883330, -99.900000)"
4,Achiras,370,L6,780.0,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"


In [6]:
del csv_meteors["GeoLocation"]
csv_meteors.head()

,name,id,recclass,mass,year,reclat,reclong
0,Aachen,1,L5,21.0,1880.0,50.77500,6.08333
1,Aarhus,2,H6,720.0,1951.0,56.18333,10.23333
2,Abee,6,EH4,107000.0,1952.0,54.21667,-113.00000
3,Acapulco,10,Acapulcoite,1914.0,1976.0,16.88333,-99.90000
4,Achiras,370,L6,780.0,1902.0,-33.16667,-64.95000


In [7]:
# Check for incomplete rows
csv_meteors.count()

name        45716
id          45716
recclass    45716
mass        45585
year        45428
reclat      38401
reclong     38401
dtype: int64

In [8]:
# Rename columns
csv_meteors_new = csv_meteors.rename(columns={'recclass':'classification', 'reclat':'latitude', 'reclong':'longitude'})
csv_meteors_new.head()

,name,id,classification,mass,year,latitude,longitude
0,Aachen,1,L5,21.0,1880.0,50.77500,6.08333
1,Aarhus,2,H6,720.0,1951.0,56.18333,10.23333
2,Abee,6,EH4,107000.0,1952.0,54.21667,-113.00000
3,Acapulco,10,Acapulcoite,1914.0,1976.0,16.88333,-99.90000
4,Achiras,370,L6,780.0,1902.0,-33.16667,-64.95000


In [9]:
# Drop rows with null or 0 value in column                         
csv_meteors_new = csv_meteors_new[csv_meteors_new.latitude != 0].dropna()
csv_meteors_new = csv_meteors_new[csv_meteors_new.longitude != 0].dropna()
csv_meteors_new = csv_meteors_new[csv_meteors_new.mass != 0].dropna()
csv_meteors_new = csv_meteors_new[csv_meteors_new.year != 0].dropna()

csv_meteors_new.count()

name              31688
id                31688
classification    31688
mass              31688
year              31688
latitude          31688
longitude         31688
dtype: int64

In [10]:
# Delete all rows with mass of less than 100
csv_meteors_new2 = csv_meteors_new[csv_meteors_new['mass'] >=100]
csv_meteors_new2.count()

name              10616
id                10616
classification    10616
mass              10616
year              10616
latitude          10616
longitude         10616
dtype: int64

In [11]:
csv_meteors_new2.to_csv (r'C:/Users/jariwalaj2/Desktop/Project-2-VisualizeMe/Data/data.csv', index = False, header = True)


In [12]:
csv_meteors_new3 = pd.read_csv(r'C:/Users/jariwalaj2/Desktop/Project-2-VisualizeMe/Data\data.csv')
csv_meteors_new3.to_json(r'C:/Users/jariwalaj2/Desktop/Project-2-VisualizeMe/landings.json')

In [16]:
json_file = pd.read_json(r'C:/Users/jariwalaj2/Desktop/Project-2-VisualizeMe/landings.json')

json_file

,name,id,classification,mass,year,latitude,longitude
0,Aarhus,2,H6,720.0,1951,56.18333,10.23333
1,Abee,6,EH4,107000.0,1952,54.21667,-113.00000
2,Acapulco,10,Acapulcoite,1914.0,1976,16.88333,-99.90000
3,Achiras,370,L6,780.0,1902,-33.16667,-64.95000
4,Adhi Kot,379,EH4,4239.0,1919,32.10000,71.80000
...,...,...,...,...,...,...,...
10611,Zhongxiang,30406,Iron,100000.0,1981,31.20000,112.50000
10612,Zillah 001,31355,L6,1475.0,1990,29.03700,17.01850
10613,Zillah 002,31356,Eucrite,172.0,1990,29.03700,17.01850
10614,Zubkovsky,31357,L6,2167.0,2003,49.78917,41.50460


In [ ]:
csv_meteors['year'] = csv_meteors.dropna()['year'].astype(np.int64)

In [ ]:
# Plot a scatter graph to determine how the landings increased over years.

years = csv_meteors.year.unique()
landings = []

for year in years:
    landings.append(len(csv_meteors[csv_meteors.year == year]))

s = [x for x in landings]

plt.scatter(years, landings, s=s, c='r', edgecolors='black')
plt.xlabel('Year')
plt.xlim(xmin=800)
plt.xlim(xmax=2200)
plt.ylabel('Landings')
plt.ylim(ymin=0)
plt.ylim(ymax=3500)
    

In [ ]:
# Let's take a look at the number of discoveries as a function of the latitude

plt.scatter(csv_meteors.year, csv_meteors.reclat,color='g',alpha=0.4)
plt.xlim(1800,2020)
plt.xlim(xmax=2020)
plt.ylim(-90,90)
plt.ylabel('Latitude')
plt.xlabel('Year')
plt.title('Meteorite recorded Latitude vs Year')